## <center> Content-based recommender models </center>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import collections
import numpy as np
import os
import pandas as pd
import torch
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from utils.data import MatchDataGenerator, df_to_dict
from utils.basic_layers import MLP, EmbeddingLayer
from utils.features import SparseFeature, SequenceFeature
from utils.match import Annoy, generate_seq_feature_match, gen_model_input
from utils.metrics import topk_metrics
from utils.trainer import MatchTrainer

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
torch.manual_seed(42);

**Ключевая идея** - давайте попробуем учиться не только на взаимодействиях users и items, но и добавлять в модель имеющиеся по ним признаки.

Способов немало, но сегодня разберем наиболее интересные и распространенные примеры:
- DSSM и ее адаптации
- LightFM

#### DSSM (Deep Structured Semantic Model) 

Вспомним, как выглядела оригинальная модель DSSM для задачи ранжирования выдачи поиска.  

<img src='https://kishorepv.github.io/images/DSSM_layers.png' width=700>

где $R(Q, D) = cosine(y_{Q}, y_{D}) = \frac{y_{Q}^{T} y_{D}}{||y_{Q}|| \cdot ||y_{D}||}$. 

При этом, считаем оценку семантической релевантности текущего документа и заданном запроса через как:

$$P(D|Q) = \frac{exp(\gamma R(Q, D)}{\sum_{D' \in D}exp(\gamma R(Q, D') }$$

Проблема со знаменателем давно известна, поэтому минимизировать будем следующую функцию:

$$L (Λ) = - log \prod_{(Q, D^{+})} P(D^{+}|Q)$$

Слева - уже знакомая нам постановка задачи для матричной факторизации. Справа - как можем модифицировать DSSM под решение задачи рекомендаций и с обогащением признаками.

<img src='images/dot_prod.png' width=600>

In [2]:
"""
References: 
    paper: (CIKM'2013) Learning Deep Structured Semantic Models for Web Search using Clickthrough Data
    url: https://posenhuang.github.io/papers/cikm2013_DSSM_fullversion.pdf
    code: https://github.com/bbruceyuan/DeepMatch-Torch/blob/main/deepmatch_torch/models/dssm.py
"""

class DSSM(torch.nn.Module):
    """Deep Structured Semantic Model
    Args:
        user_features (list[Feature Class]): training by the user tower module.
        item_features (list[Feature Class]): training by the item tower module.
        temperature (float): temperature factor for similarity score, default to 1.0.
        user_params (dict): the params of the User Tower module, 
        keys include:`{"dims":list, "activation":str, "dropout":float, "output_layer":bool`}.
        item_params (dict): the params of the Item Tower module, keys include:`{"dims":list, "activation":str, "dropout":float, "output_layer":bool`}.
    """

    def __init__(self, user_features, item_features, user_params, item_params, temperature=1.0):
        super().__init__()
        self.user_features = user_features
        self.item_features = item_features
        self.temperature = temperature
        self.user_dims = sum([f.embed_dim for f in user_features])
        self.item_dims = sum([f.embed_dim for f in item_features])

        self.embedding = EmbeddingLayer(user_features + item_features)
        self.user_mlp = MLP(self.user_dims, output_layer=False, **user_params)
        self.item_mlp = MLP(self.item_dims, output_layer=False, **item_params)
        self.mode = None

    def forward(self, x):
        user_embedding = self.user_tower(x)
        item_embedding = self.item_tower(x)
        if self.mode == "user":
            return user_embedding
        if self.mode == "item":
            return item_embedding
        y = # YOUR ONE-LINE CODE HERE
        return torch.sigmoid(y)

    def item_tower(self, x):
        if self.mode == "user":
            return None
        # Какая тут размерность? 
        input_item = self.embedding(x, self.item_features, squeeze_dim=True)
        item_embedding = self.item_mlp(input_item)
        item_embedding = F.normalize(item_embedding, p=2, dim=1)
        return item_embedding
    
    def user_tower(self, x):
        if self.mode == "item":
            return None
        input_user = self.embedding(x, self.user_features, squeeze_dim=True)
        user_embedding = self.user_mlp(input_user)
        user_embedding = F.normalize(user_embedding, p=2, dim=1)
        return user_embedding

Кстати, recap, как называется архитектура с картинки ниже?

Сравните ее еще раз с архитектурами DSSM и CF (ч.с. MF) и не путайте =)

<img src='images/NCF.png' width=400>

Есть много имплементаций DSSM, можно посмотреть, например, в библиотеке [RecBole](https://github.com/RUCAIBox/RecBole/blob/4b6c6fef9b2f21326876f81e5d76631b280b0909/recbole/model/context_aware_recommender/dssm.py)

<img src='images/dot_prod.png' width=600>

Как будем оптимизировать для RecSys? Лоссы можно выбирать разные. Самые простые из них:

* **Кросс-энтропия**

$\hat p_{ui} = \sigma (R(u, i)) = \sigma(y_{u}^T y_{i})$ - вероятность, что юзер совершит действие с айтемом.

Тогда функция потерь:

$L = - \sum_{u, i} (r_{ui}\cdot log \hat p_{ui} + (1 - r_{ui}) \cdot log(1 - \hat p_{ui}))$

* **Triplet loss**

Рассматриваем тройки из пользователя, положительного примера для него и отрицательного - $R(u, i_{+}, i_{-})$

$L(R(u, i_{+}), R(u, i_{-})) = max(0, \alpha - R(u, i_{+}) + R(u, i_{-}))$

То есть, пытаемся увеличивать разницу, задавая некоторый порог (отступ) альфа. 

* Давайте тут остановимся и подумаем, какие варианты negative sampling при оптимизации лосса тут можно использвать?

* Какие признаки можно использовать по users, items в качестве input данных? 

* Какие преимущества и недостатки у DSSM модели?


| Метод | Достоинства  |  Недостатки | 
|---|---|---|
| DSSM| Может выявить нелинейные закономерности  |  Дополнительная работа с признаками и оценкой их вклада|  
|
|| Real-time inference на проде | Необходимость подбора параметров и тюнинга гиперпараметров |  
|
|| Возможность добавлять признаки разного типа| Уже вряд ли можно назвать SOTA по качеству ранжирования | 
|
|| Подходит как мета-алгоритм для каскадной модели| |  
|
|| User и item эмбеддинги в едином векторном пространстве| |  

### MovieLens-1M

Источник: https://grouplens.org/datasets/movielens/1m/ с данными в трех файлах: users.dat, ratings.dat, movies.dat, которые мы объединились в один csv. 
Для быстроты вычислений на семинаре мы взяли первые 10к строк из датасета, но если будете воспроизводить у себя и захотите посмотреть на финальное качество модели, попробуйте поработать со всем датасетом. 

In [3]:
file_path = 'ml-1m.csv'
data = pd.read_csv(file_path, nrows=10000)
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


Препроцессинг - энкодинг, обработка колонки с жанрами (оставим только первый указанный), сделаем split по числу интеракций в тесте.

In [4]:
data["cat_id"] = data["genres"].apply(lambda x: x.split("|")[0])
user_col, item_col = "user_id", "movie_id"
sparse_features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip', "cat_id"]

In [5]:
save_dir = './saved/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
print(f'Before encoding: \n {data[sparse_features].tail()}')

feature_max_idx = {}
for feature in sparse_features:
    encoder = LabelEncoder()
    data[feature] = encoder.fit_transform(data[feature]) + 1 # лучше энкодить не с 0, особенно в sequential NN
    feature_max_idx[feature] = data[feature].max() + 1
    if feature == user_col:
        user_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(encoder.classes_)}
    if feature == item_col:
        item_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(encoder.classes_)}
np.save(save_dir + "raw_id_maps.npy", (user_map, item_map))

print(f'After encoding: \n {data[sparse_features].tail()}')

Before encoding: 
       user_id  movie_id gender  age  occupation   zip  cat_id
9995      180      1193      M   45          12  1603   Drama
9996      180      3408      M   45          12  1603   Drama
9997      180       608      M   45          12  1603   Crime
9998      180      3068      M   45          12  1603   Drama
9999      180      3578      M   45          12  1603  Action
After encoding: 
       user_id  movie_id  gender  age  occupation  zip  cat_id
9995       46       644       2    5          11    2       8
9996       46      1881       2    5          11    2       8
9997       46       365       2    5          11    2       6
9998       46      1709       2    5          11    2       8
9999       46      1983       2    5          11    2       1


In [6]:
user_cols = ["user_id", "gender", "age", "occupation", "zip"]
item_cols = ['movie_id', "cat_id"]
user_profile = data[user_cols].drop_duplicates('user_id')
item_profile = data[item_cols].drop_duplicates('movie_id')

Наиболее интересные тут параметры - `sample_method`, `mode`, `neg_ratio`, `min_item`. 

In [7]:
df_train, df_test = generate_seq_feature_match(data,
                                               user_col,
                                               item_col,
                                               time_col="timestamp",
                                               item_attribute_cols=[],
                                               sample_method=1,
                                               mode=0,
                                               neg_ratio=3,
                                               min_item=3)

generate sequence features: 100%|████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 312.81it/s]

n_train: 39632, n_test: 46
0 cold start users droped 


Важно помнить про `max sequence length`.

In [8]:
x_train = gen_model_input(df_train, user_profile, user_col, item_profile, item_col, seq_max_len=50)
x_test = gen_model_input(df_test, user_profile, user_col, item_profile, item_col, seq_max_len=50)
y_train = x_train["label"]
y_test = x_test["label"]

{k: v[:3] for k, v in x_train.items()}

{'user_id': array([17, 12, 16]),
 'movie_id': array([1600, 1529, 2105]),
 'hist_movie_id': array([[1936,  433, 1087,  409, 1553,  767,  755,  764,  195,  618, 2044,
         2166, 1557, 1597, 1682, 2098,  234,  765, 1099, 2157,  343,  759,
          146, 1815, 1217, 1971, 1205, 1084, 1555, 1561, 1100, 1517, 1409,
          323,  706, 1181, 2023, 1287,  107,  761,  113, 1384, 2158, 1093,
         2159,  406,  490,  632, 1565, 1101],
        [2053, 1168, 1415, 1339, 1418, 1105, 2016, 1034,  415, 1468, 1276,
         1106, 2092, 1012, 1690, 1179,   46,  860,  976, 2169,  112,  566,
         1123,  185, 1161,  754,  231,  783, 1664,  878,  117,  877, 2086,
          103, 1469,  104, 1352, 1382,  996,  463,  411, 2108,  686,  663,
          492,  645, 1921,  616,  872,  550],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

In [9]:
user_features = [
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in user_cols
]

user_features += [
    SequenceFeature("hist_movie_id",
                    vocab_size=feature_max_idx["movie_id"],
                    embed_dim=16,
                    pooling="mean",
                    shared_with="movie_id")
]

item_features = [
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in item_cols
]

In [10]:
all_item = df_to_dict(item_profile)
test_user = x_test
data_generator = MatchDataGenerator(x=x_train, y=y_train)
train_dl, test_dl, item_dl = data_generator.generate_dataloader(test_user, all_item, batch_size=128)

In [ ]:
model = DSSM(user_features,
             item_features,
             temperature=0.02,
             user_params={
                 "dims": [256, 128, 64],
                 "activation": 'prelu',
             },
             item_params={
                 "dims": [256, 128, 64],
                 "activation": 'prelu',
             })

trainer = MatchTrainer(model,
                       mode=0,
                       optimizer_params={
                           "lr": 1e-2,
                           "weight_decay": 1e-5
                       },
                       n_epoch=3,
                       device='cpu',
                       model_path=save_dir)


trainer.fit(train_dl)

epoch: 0


train: 100%|█████████████████████████████████████████████████████████████████████████████████████| 310/310 [00:06<00:00, 51.51it/s, loss=0.56]


epoch: 1


train:   8%|███████▏                                                                             | 26/310 [01:13<13:25,  2.84s/it, loss=0.569]

Как вы думаете, зачем нам понадобился ANN алгоритм? 

In [ ]:
def match_evaluation(user_embedding, item_embedding, test_user, all_item, user_col='user_id', item_col='movie_id',
                     raw_id_maps="./raw_id_maps.npy", topk=10):
    
    # Fit Annoy tree on item embeddings
    annoy = Annoy(n_trees=10)
    annoy.fit(item_embedding)

    # For each user get top-k similar items
    user_map, item_map = np.load(raw_id_maps, allow_pickle=True)
    match_res = collections.defaultdict(dict)
    for user_id, user_emb in zip(test_user[user_col], user_embedding):
        items_idx, items_scores = annoy.query(v=user_emb, n=topk)
        match_res[user_map[user_id]] = np.vectorize(item_map.get)(all_item[item_col][items_idx])

    # Get ground truth
    data = pd.DataFrame({user_col: test_user[user_col], item_col: test_user[item_col]})
    data[user_col] = data[user_col].map(user_map)
    data[item_col] = data[item_col].map(item_map)
    user_pos_item = data.groupby(user_col).agg(list).reset_index()
    ground_truth = dict(zip(user_pos_item[user_col], user_pos_item[item_col]))

    # Compute top-k metrics
    out = topk_metrics(y_true=ground_truth, y_pred=match_res, topKs=[topk])
    return out

In [ ]:
user_embedding = trainer.inference_embedding(model=model, mode="user", data_loader=test_dl, model_path=save_dir)
item_embedding = trainer.inference_embedding(model=model, mode="item", data_loader=item_dl, model_path=save_dir)
match_evaluation(user_embedding, item_embedding, test_user, all_item, topk=100, raw_id_maps=save_dir + "raw_id_maps.npy")

### YouTubeDNN

Covington, P., Adams, J. and Sargin, E., 2016, September. Deep neural networks for youtube recommendations. In Proceedings of the 10th ACM conference on recommender systems (pp. 191-198).

https://dl.acm.org/doi/pdf/10.1145/2959100.2959190

<img src='images/YoutubeDNN.png' width=800>

In [ ]:
class YoutubeDNN(torch.nn.Module):
    """
    The match model mentioned in `Deep Neural Networks for YouTube Recommendations` paper.
    It's a DSSM match model trained by global softmax loss on list-wise samples. 
    In origin paper, item dnn tower is missing.
    Args:
        user_features (list[Feature Class]): training by the user tower module.
        item_features (list[Feature Class]): training by the embedding table, it's the item id feature.
        neg_item_feature (list[Feature Class]): training by the embedding table, it's the negative items id feature.
        user_params (dict): the params of the User Tower module, 
        keys include:`{"dims":list, "activation":str, "dropout":float, "output_layer":bool`}.
        temperature (float): temperature factor for similarity score, default to 1.0.
    """

    def __init__(self, user_features, item_features, neg_item_feature, user_params, temperature=1.0):
        super().__init__()
        self.user_features = user_features
        self.item_features = item_features
        self.neg_item_feature = neg_item_feature
        self.temperature = temperature
        self.user_dims = sum([fea.embed_dim for fea in user_features])
        self.embedding = EmbeddingLayer(user_features + item_features)
        self.user_mlp = MLP(self.user_dims, output_layer=False, **user_params)
        self.mode = None

    def forward(self, x):
        user_embedding = self.user_tower(x)
        item_embedding = self.item_tower(x)
        if self.mode == "user":
            return user_embedding
        if self.mode == "item":
            return item_embedding

        y = torch.mul(user_embedding, item_embedding).sum(dim=2)
        y = y / self.temperature
        return y

    def user_tower(self, x):
        if self.mode == "item":
            return None
        # [batch_size, num_features * deep_dims]
        input_user = self.embedding(x, self.user_features, squeeze_dim=True)
        # [batch_size, 1, embed_dim]
        user_embedding = self.user_mlp(input_user).unsqueeze(1)
        user_embedding = F.normalize(user_embedding, p=2, dim=2)
        if self.mode == "user":
            return user_embedding.squeeze(1)
        return user_embedding

    def item_tower(self, x):
        if self.mode == "user":
            return None
        #[batch_size, 1, embed_dim]
        pos_embedding = self.embedding(x, self.item_features, squeeze_dim=False)
        pos_embedding = F.normalize(pos_embedding, p=2, dim=2)
        # inference embedding mode
        if self.mode == "item":
            # [batch_size, embed_dim]
            return pos_embedding.squeeze(1)
        #[batch_size, n_neg_items, embed_dim]
        neg_embeddings = self.embedding(x, self.neg_item_feature,
                                        squeeze_dim=False).squeeze(1)
        neg_embeddings = F.normalize(neg_embeddings, p=2, dim=2)
        # [batch_size, 1 + n_neg_items, embed_dim]
        return torch.cat((pos_embedding, neg_embeddings), dim=1)

In [ ]:
df_train, df_test = generate_seq_feature_match(data,
                                               user_col,
                                               item_col,
                                               time_col="timestamp",
                                               item_attribute_cols=[],
                                               sample_method=1,
                                               mode=2,
                                               neg_ratio=3,
                                               min_item=0)
x_train = gen_model_input(df_train, user_profile, user_col, item_profile, item_col, seq_max_len=50)
y_train = np.array([0] * df_train.shape[0])
x_test = gen_model_input(df_test, user_profile, user_col, item_profile, item_col, seq_max_len=50)

user_cols = ['user_id', 'gender', 'age', 'occupation', 'zip']

user_features = [SparseFeature(name, vocab_size=feature_max_idx[name], embed_dim=16) for name in user_cols]
user_features += [
    SequenceFeature("hist_movie_id",
                    vocab_size=feature_max_idx["movie_id"],
                    embed_dim=16,
                    pooling="mean",
                    shared_with="movie_id")
]

item_features = [SparseFeature('movie_id', vocab_size=feature_max_idx['movie_id'], embed_dim=16)]
neg_item_feature = [
    SequenceFeature('neg_items',
                    vocab_size=feature_max_idx['movie_id'],
                    embed_dim=16,
                    pooling="concat",
                    shared_with="movie_id")
]

all_item = df_to_dict(item_profile)
test_user = x_test

dg = MatchDataGenerator(x=x_train, y=y_train)
train_dl, test_dl, item_dl = dg.generate_dataloader(test_user, all_item, batch_size=512)



In [ ]:
model = YoutubeDNN(user_features, item_features, neg_item_feature, 
                   user_params={"dims": [128, 64, 16]}, temperature=0.02)

trainer = MatchTrainer(model,
                       mode=2,
                       optimizer_params={
                           "lr": 1e-2,
                           "weight_decay": 1e-5
                       },
                       n_epoch=1,
                       device='cpu',
                       model_path=save_dir)

trainer.fit(train_dl)

print("inference embedding")
user_embedding = trainer.inference_embedding(model=model, mode="user", data_loader=test_dl, model_path=save_dir)
item_embedding = trainer.inference_embedding(model=model, mode="item", data_loader=item_dl, model_path=save_dir)
match_evaluation(user_embedding, item_embedding, test_user, all_item, topk=100, raw_id_maps="./saved/raw_id_maps.npy")






### LightFM. 

Kula, M., 2015. Metadata embeddings for user and item cold-start recommendations. arXiv preprint arXiv:1507.08439.

http://ceur-ws.org/Vol-1448/paper4.pdf?ref=https://githubhelp.com

Нейросетевые модели - не всегда самая лучшая опция, в частности, есть довольно сильная модель LightFM, которая обобщает коллабративную фильтрацию для случая с добавлением признаков. 


* Это гибридный подход коллаборативной фильтрации и контентной модели, которая предсталвяет эмбеддинги пользователей и эмбеддинги объектов как линейные комбинации из обученных векторов известных признаков - т.е. суммы новых латентных признаков. При этом, это позволяет обучать модель как в режиме без признаков, так и с ними, решая проблему холодного старта (т.к. по новым пользователям и объектам можно использовать их признаки) и проблему слишком разреженных данных (high sparsity problem). Таким образом, LightFM умеет хорошо работает как с плотными, так и с разреженными данными, и, как бонус, кодировать в эмбеддингах признаков семантическую информацию по аналогии с подходами для получения эмбеддингов слов (например, w2v).


*  Формализация.  <br> $U$ - множество пользователей, <br> $I$ - множество объектов, <br> $F^{U}$ - множество признаков пользователей, <br> $F^{I}$- множество признаков объектов. <br>
Все пары $(u, i) \in U × I$ - это объединение всех положительных $S^{+}$ и отрицательных $S^{-}$ интеракций. 

Каждый пользователь описан набором заранее известных признаков (мета данных) $f_u \subset F^U$, то же самое для объектов  $f_i \subset F^I$.

Латентное представление пользователя представлено суммой его латентных векторов признаков: 

$$q_u = \sum_{j \in f_u} e^U_j$$

Аналогично для объектов: $$p_u = \sum_{j \in f_i} e^I_j$$

Так же, по пользователю и объекту есть смещения (bias): 

$$b_u = \sum_{j \in f_u} b^U_j$$

$$b_i = \sum_{j \in f_i} b^I_j$$

Предсказание из модели будет получать через скалярное произведение эмбедингов пользователя и объекта. 

$$\hat r_{ui} = f (q_u \cdot p_i + b_u + b_i)$$

Функция f() может быть разной, автор статьи выбрал сигмоиду, поскольку использовал бинарные данные.

$$f(x) = \frac{1}{1 + exp(-x)}$$

Задача оптимизации будет сформулирована как максимизация правдоподобия (данных при параметрах), с обучением модели с помощью стохастического градиентного спуска.

$$L(e^U, e^I, b^U, b^I) = \prod_{(u, i) \in S^+} \hat r_{ui} \cdot  \prod_{(u, i) \in S^-} (1 - \hat r_{ui})$$

Источники:

1. Исходная статья DSSM https://posenhuang.github.io/papers/cikm2013_DSSM_fullversion.pdf 
2. RecBole https://github.com/RUCAIBox/RecBole
3. https://github.com/datawhalechina/torch-rechub 
4. Статья LightFM http://ceur-ws.org/Vol-1448/paper4.pdf?ref=https://githubhelp.com
5. Отличная **библиотека от автора статьи** c примерами запусков - https://github.com/lyst/lightfm 